# Exercise 3: Langchain agents

LangChain provides [various kinds of agents](https://python.langchain.com/docs/modules/agents/agent_types/). These agents have the capability to initiate actions and autonomously decide when and in what sequence to perform them. This enables users to rapidly and effectively implement a reasoning agent for a range of different contexts.

In this exercise we’ll examine a few of the most common.

In [ ]:
# autoreload imports
%load_ext autoreload

%autoreload 2

In [ ]:
from llm_in_production.openai_utils import get_openai_client

from langchain import hub
from langchain.agents import load_tools, create_openai_tools_agent, AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import requests
import os
import pandas as pd

# Here we create the client.
client = get_openai_client(use_langchain=True, model_name=os.environ["GPT_4_MODEL_NAME"], temperature=0)

## Tools

We'll start by creating some tools the LangChain agents can use.

LangChain has a number of inbuilt tools. For example,
- `arxiv`: A search tool for papers on the [arXiv](https://arxiv.org) website
- `dalle-image-generator`: A tool for creating images with Dall-E

In [ ]:
tools = load_tools(
    ["arxiv", 
    #  "dalle-image-generator"
     ],
)

In [ ]:
tools

Alternatively, we can conveniently create custom tools with the [`@tools` decorator](https://python.langchain.com/docs/modules/agents/tools/custom_tools#tool-decorator).

In [ ]:
@tool
def get_current_weather(location: str) -> dict:
    """Get the current weather conditions in a given location

    :param location: The city (and state), e.g. "San Francisco, CA"
    :return: The weather conditions
    """
    return requests.get(
        f"https://api.weatherapi.com/v1/current.json?key={os.environ['WEATHER_API_KEY']}&q={location}"
    ).json()

In [ ]:
print(get_current_weather.name)
print(get_current_weather.description)
print(get_current_weather.args)

In [ ]:
tools.append(get_current_weather)
tools

## OpenAI Tool calling

One of the more simple agents we can create with LangChain is an Agent that uses OpenAI Tool calling.

As we have seen, OpenAI models have now been fine-tuned to detect when one or more function(s) should be called and respond with the inputs that should be passed to the function(s).

To initialise LangChain's OpenAI Tool Calling agent we require:
- an LLM
- Tools
- a Prompt to guide the agent (we will use the default prompt for tool calling)

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt.messages[0].prompt.template)

agent = create_openai_tools_agent(client, tools, prompt)

The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor.

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now run the agent on a few queries! Note that for now, these are all stateless queries (it won’t remember previous interactions).

In [ ]:
input = {"input": "Is it raining in Paris?"}

response = agent_executor.invoke(input)
response['output']

### Exercise 3a: ReAct paper

Use the agent that we've created to find out which paper initially introduced the idea of ReAct prompting for LLMs?

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

## ReAct

ReAct can also be implemented easily with LangChain.

Again we need an LLM, tools and a prompt to implement the agent.

In [ ]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)
agent = create_react_agent(client, tools, prompt)

We can then create an AgentExecutor and invoke a response to an input prompt.



In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True, handle_parsing_errors=True)

In [ ]:
input = {"input": "Is it raining in Paris?"}

response = agent_executor.invoke(input)
response['output']

In [ ]:
print(response['intermediate_steps'][0][0].log)

## Pandas Agent

LangChain has a [libary of agents that are optimised for specific tasks](https://python.langchain.com/docs/integrations/toolkits). For example, the Pandas agent.

Let's demonstrate the Pandas agent on the Chickweight dataset.

In [ ]:
chickweight = pd.read_csv("chickweight.csv")

The agent requires a model and a DataFrame.

In [ ]:
agent = create_pandas_dataframe_agent(client, chickweight, verbose=True, return_intermediate_steps=True)

We can invoke queries directly to the agent.

In [ ]:
response = agent.invoke("How many columns are there?")
response['output']

In [ ]:
response = agent.invoke("What is the maximum value in the weight column?")
response['output']

In [ ]:
response = agent.invoke("Can you return the weight column as a list?")
response['output']

---